# 「シゴトがはかどるPython自動処理の教科書」 クジラ飛行机 著 (つづき)

## Webブラウザの自動化／スクレイピング

**Requestsモジュールを使ってダウンロードしよう**

In [2]:
import requests

url = 'https://api.aoikujira.com/time/get.php'
result = requests.get(url)
print(result.text)

2023/10/20 09:46:25


In [3]:
import requests

url = 'https://api.aoikujira.com/time/get.php'
result = requests.get(url)
print('ok=', result.ok)
if result.ok:
    print('text=', result.text)
    print('status_code=', result.status_code)

ok= True
text= 2023/10/20 09:50:44
status_code= 200


**画像ファイルをダウンロードしてみよう**

In [6]:
import requests

url = 'https://uta.pw/shodou/img/3/3.png'
res = requests.get(url)
if not res.ok:
    print('失敗:', res.status_code)
    quit()
with open('gyudon.png', 'wb') as fp:
    fp.write(res.content)
print('ok.')

ok.


for文と組み合わせて連続で画像をダウンロード

In [8]:
import requests
import os, time

save_dir = 'img_chap4/'
base_url = 'https://uta.pw/shodou/img/{0}/{1}.png'

def download_all():
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    for id in range(1, 11):
        download_file(id)
        time.sleep(1)

def download_file(id):
    url = base_url.format(id%31, id)
    save_file = save_dir + str(id) + '.png'
    res = requests.get(url)
    if not res.ok:
        print('失敗:', res.status_code)
        return
    with open(save_file, 'wb') as fp:
        fp.write(res.content)
    print('save:', save_file)

if __name__ == '__main__':
    download_all()

save: img_chap4/1.png
save: img_chap4/2.png
save: img_chap4/3.png
save: img_chap4/4.png
save: img_chap4/5.png
save: img_chap4/6.png
save: img_chap4/7.png
save: img_chap4/8.png
save: img_chap4/9.png
save: img_chap4/10.png


**スクレイピング -- BeautifulSoup を使おう**

In [5]:
from bs4 import BeautifulSoup

with open('html_chap4/fish.html', encoding='utf-8') as fp:
    html_str = fp.read()

soup = BeautifulSoup(html_str, 'html5lib')
title = soup.find('title')
print(title.text)

魚について


複数の要素を検索するときは、find_allメソッド

In [9]:
fishes = soup.find_all('h2')
for e in fishes:
    print(e.text)

カツオ
ウナギ
サケ


ウナギの値段を調べる

In [11]:
for h2 in soup.find_all('h2'):
    if h2.string == 'ウナギ':
        for e in h2.next_siblings:
            if e.name == 'p':
                if e['class'][0] == 'price':
                    print(e.string)

12,800円/kg


In [21]:
unagi = soup.find(id='eel')
unagi_price = unagi.find(class_ = 'price')
print(unagi_price.string)

12,800円/kg


In [28]:
p = soup.select('div#eel > p.price')
print(p[0].text)

12,800円/kg


魚と解説と値段を列挙する

In [34]:
from bs4 import BeautifulSoup

with open('html_chap4/fish.html', encoding='utf-8') as fp:
    html_str = fp.read()
soup = BeautifulSoup(html_str, 'html5lib')

res = []
div_list = soup.select('#fishes > div')
for div in div_list:
    fish = div.h2.text
    desc = div.select('.desc')[0].text
    price = div.select('.price')[0].text
    print(fish, desc, price)
    res.append([fish, desc, price])

import csv
with open('html_chap4/fish.csv', 'wt', encoding='sjis', newline='') as fp:
    csv.writer(fp).writerows(res)

カツオ 大型肉食魚。鉄分が多く味の個性が強い。 2,980円/kg
ウナギ 海で産卵し川に戻る。蒲焼きで食べる。 12,800円/kg
サケ 川で産卵し海で過ごす。脂のりが良い。 1,890円/kg


**ページ内のリンクを集めて一気にダウンロードしよう**

1. 名作作品のページのHTMLを取得
2. BeautifulSoupで解析して画像のURLを取得
3. 取得したURLの画像を連続でダウンロード

In [ ]:
import os, time, requests, urllib
from bs4 import BeautifulSoup

target_url = 'https://uta.pw/shodou/index.php?master'
save_dir = 'img_chap4/meisaku/'

# ダウンロードのメイン処理
def download_images():
    html = requests.get(target_url).text
    urls = get_image_urls(html)
    go_download(urls)

# HTMLから画像のURL一覧を取得
def get_image_urls(html):
    soup = BeautifulSoup(html, 'html5lib')
    res = []
    for img in soup.find_all('imag'):
        src = img['src'] # 相対パス
        url = urllib.parse.urljoin(target_url, src) #絶対パスに変換
        print('img.src=', url)
        res.append(url)
    return res

# 連続でURL一覧をダウンロード
def go_download(urls):
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    for url in urls:
        fname = os.path.basename(url)
        save_file = save_dir + fname
        r = requests.get(url)
        with open(save_file, 'wb') as fp:
            fp.write(r.content)
            print('save:', save_file)
        time.sleep(1)

if __name__ == '__main__':
    download_images()

**相対URLを絶対URLに変換するurljoin関数**

```絶対URL = urllib.parse.urljoin(基本URL, 相対URL)```

In [45]:
from urllib.parse import urljoin

urljoin('https://example.com/a/b/c.html', 'hoge.jpg')

'https://example.com/a/b/hoge.jpg'

In [47]:
urljoin('https://example.com/a/b/c.html', '../fuga.jpg')

'https://example.com/a/fuga.jpg'

In [49]:
urljoin('https://example.com/a/b/c.html', '../../img/foo.jpg')

'https://example.com/img/foo.jpg'

In [50]:
urljoin('https://example.com/a/b/c.html', '/logo.jpg')

'https://example.com/logo.jpg'

**サイトリンクをたどって丸ごと資料を取得しよう**

Python公式ドキュメント、チュートリアルページのダウンロード

処理の手順

1. 基本となるページを取得(https://doc.python.org/ja/3/tutorial/index.html)
2. リンクされているURLを列挙して以下繰り返す
3. そのリンクがチュートリアル外なら何もしない
4. そのリンクがすでに取得済みであれば何もしない
5. HTMLをファイルに保存
6. 取得したページからリンクされているURLを列挙して3.に戻る

In [51]:
import requests, urllib, os, time
from bs4 import BeautifulSoup

save_dir = './pydoc_tutorial/'
top_page = 'https://doc.python.org/ja/3/tutorial/index.html'
check_url = 'https://doc.python.org/ja/3/tutorial/'
visited ={}

def get_page(url): # 指定されたURLのページを取得する
    if check_url not in url:
        return
    if url in visited:
        return
    visited[url] = True # 訪問済にする
    res = requests.get(url)
    res.encoding = res.apparent_encoding
    html = res.text
    fname = save_dir + os.path.basename(url)
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    with open(fname, 'wt', encoding='utf-8') as f:
        f.write(html)
        print('save:',fname)
    time.sleep(1)
    soup = BeautifulSoup(html, 'html5lib')
    for a in soup.find_all('a'):
        a_url = urllib.parse.urljoin(url, a['href'])
        a_url = urllib.parse.urldefrag(a_url)[0]
        get_page(a_url)

if __name__ == '__main__':
    get_page(top_page)

save: ./pydoc_tutorial/index.html
save: ./pydoc_tutorial/appetite.html
save: ./pydoc_tutorial/interpreter.html
save: ./pydoc_tutorial/introduction.html
save: ./pydoc_tutorial/controlflow.html
save: ./pydoc_tutorial/datastructures.html
save: ./pydoc_tutorial/modules.html
save: ./pydoc_tutorial/inputoutput.html
save: ./pydoc_tutorial/errors.html
save: ./pydoc_tutorial/classes.html
save: ./pydoc_tutorial/stdlib.html
save: ./pydoc_tutorial/stdlib2.html
save: ./pydoc_tutorial/venv.html
save: ./pydoc_tutorial/whatnow.html
save: ./pydoc_tutorial/interactive.html
save: ./pydoc_tutorial/floatingpoint.html
save: ./pydoc_tutorial/appendix.html


**Webブラウザを自動操縦しよう**  
**--ライブラリのインストール編**

SeleniumとChromeDriverのインストール

Chromeを起動してPythonのページを表示するプログラムの動作確認

In [1]:
from selenium import webdriver
import time

driver = webdriver.Chrome()
driver.get('https://python.org')
time.sleep(30)
driver.quit()

任意のページのスクリーンショットを撮影するプログラム

In [2]:
from selenium import webdriver

driver = webdriver.Chrome()
driver.get('https:ojizou003.com')
driver.save_screenshot('./img_chap4/ojizou003.png')
driver.quit()

**Webブラウザを自動操縦しよう**  
**--基本マスター編**

作詞掲示板の作品ページから作品タイトルと作者を取得するプログラム

In [3]:
from selenium import webdriver

driver = webdriver.Chrome()

url = 'https://uta.pw/sakusibbs/post.php?mml_id=35'
driver.get(url)
e = driver.find_element_by_class_name('posttitle')
print(e.text)
driver.close()

人はいさ 心も知らず ふるさとは 花ぞ昔の 香に匂ひける --- 百人一首 作


掲示板のトップページから名作アーカイブというリンクを探して、そのページを表示するプログラム

In [5]:
from selenium import webdriver
import time

driver = webdriver.Chrome()

driver.get('https://uta.pw/sakusibbs/')
link = driver.find_element_by_link_text('名作アーカイブ')
link.click()
time.sleep(30)
driver.close()

**フォーム送信**  
**--Google検索を自動化しよう**

In [6]:
from selenium import webdriver
import time

driver = webdriver.Chrome()

driver.get('https://www.google.co.jp/webhp?rls=ig')
el = driver.find_element_by_name('q')
el.send_keys('Pythonの教科書')
el.submit()

time.sleep(30)
driver.close()

**ヘッドレスモードで起動しよう**

In [15]:
from selenium import webdriver
import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

driver.get('https://uta.pw/sakusibbs/users.php?user_id=1')
a_list = driver.find_elements_by_css_selector('ul#mmlist li a')
for a in a_list:
    print('■',a.text)
    print(' ∟',a.get_attribute('href'))
    time.sleep(1)

driver.close()

■ サーバーメンテナンス後のテストの巻
 ∟ https://uta.pw/sakusibbs/post.php?mml_id=1384
■ どこまでも
 ∟ https://uta.pw/sakusibbs/post.php?mml_id=1232
■ テストとはテスト
 ∟ https://uta.pw/sakusibbs/post.php?mml_id=421
■ ヨモギ
 ∟ https://uta.pw/sakusibbs/post.php?mml_id=347
■ すたーとふろむ風呂
 ∟ https://uta.pw/sakusibbs/post.php?mml_id=346
■ 取り外す
 ∟ https://uta.pw/sakusibbs/post.php?mml_id=272
■ 危険な試験
 ∟ https://uta.pw/sakusibbs/post.php?mml_id=270
■ 数え歌
 ∟ https://uta.pw/sakusibbs/post.php?mml_id=245
■ 爽やかな風の中で
 ∟ https://uta.pw/sakusibbs/post.php?mml_id=206
■ リトマス紙と私
 ∟ https://uta.pw/sakusibbs/post.php?mml_id=141
■ サラダよ皿に乗れ
 ∟ https://uta.pw/sakusibbs/post.php?mml_id=127
■ ずーっとるーむ
 ∟ https://uta.pw/sakusibbs/post.php?mml_id=124
■ チキンカレーだけが世界
 ∟ https://uta.pw/sakusibbs/post.php?mml_id=121
■ 夕焼けと船
 ∟ https://uta.pw/sakusibbs/post.php?mml_id=118
■ 逃亡
 ∟ https://uta.pw/sakusibbs/post.php?mml_id=116
■ ゆうひ
 ∟ https://uta.pw/sakusibbs/post.php?mml_id=110
■ おもちゃ箱ガラガラ行進曲の歌詞
 ∟ https://uta.pw/sakusibbs/post.php?mml_id=108
■ 海辺でゆったり
 

**ページ全体のスクリーンショットを撮る方法**

In [16]:
from selenium import webdriver

url = 'https://python.org'
save_file = 'img_chap4/screenshot_full.png'

# メイン処理
def screenshot_full(url, save_file):
    w, h = get_page_size(url)
    screenshot_size(url, save_file, w, h)

# ページの幅と高さを取得する
def get_page_size(url):
    driver = webdriver.Chrome()
    driver.get(url)
    w = driver.execute_script(
        'return document.body.scrollWidth;'
    )
    h = driver.execute_script(
        'return document.body.scrollHeight;'
    )
    driver.close()
    print('page_size:', w, h)
    return(w, h)

# 指定サイズでページを保存
def screenshot_size(url, save_file, w, h):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    win_size = 'window-size=' + str(w) + ',' + str(h)
    options.add_argument(win_size)
    cap_driver = webdriver.Chrome(options = options)
    cap_driver.get(url)
    cap_driver.save_screenshot(save_file)
    cap_driver.close()

if __name__ == '__main__':
    screenshot_full(url, save_file)


page_size: 1019 2638


**会員制Webサイトからデータをダウンロード**

作詞掲示板にログインして、マイページからCSVファイルをダウンロード

In [30]:
from selenium import webdriver
import os, time

login_url = 'https://uta.pw/sakusibbs/users.php?action=login'
user_id, password = ('おぢぞう', 'ipCU12ySxl')
save_dir = 'C:/Users/sinis/programming/08_Python/Python_Jidousyorino_kyoukasyo/csv_chap4/'
save_file = save_dir + 'list.csv'

options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {'download.default_directory': save_dir})

#メイン処理
def login_download():
    driver = webdriver.Chrome(options = options)
    try_login(driver)
    link_click(driver, 'マイページ')
    time.sleep(3)
    link_click(driver, 'ダウンロード')
    for i in range(30):
        if os.path.exists(save_file):
            break
        time.sleep(1)
    driver.close()

# ログイン処理
def try_login(driver):
    driver.get(login_url)
    user = driver.find_element_by_name('username_mmlbbs6')
    user.send_keys(user_id)
    pwd = driver.find_element_by_name('password_mmlbbs6')
    pwd.send_keys(password)
    pwd.submit()

# ラベルを指定してリンクを検索しクリックする
def link_click(driver, label):
    a = driver.find_element_by_partial_link_text(label)
    a.click()

if __name__ == '__main__':
    login_download()